## Calculation of the SVI 

In this example the Standard Vegetation Index (SVI) is calculated from the Enhanced Vegetation Index (EVI) gained from satellite remote sensing. A MODIS vegetation product (MOD13Q1) with a temporal resolution of 16 days and a spatial resolution of 250m is used (https://lpdaac.usgs.gov/dataset_discovery/modis/modis_products_table/mod13q1). 

Hit `Shift + Enter` or the run button to execute the cells.

### Download and install Anaconda 3

To run this Jupyter Notebook you need python installed on your computer. We recommend to download and install Anaconda 3 (https://www.anaconda.com/download/) with the python 3.6 version as it includes a lot of usefule packages.

### Download MODIS data as Geotiffs from AppEEARS

Access the AppEEARS website and create a user account (free) for USGS in case you do not already have one. Click on __Extract__ and __Area Sample__ and start a new request.

![](http://www.un-spider.org/sites/default/files/snap1_adj.PNG)

1. Enter a request name. 
2. Define your region of interest by specifying a __ESRI shapefile__ (zip) or drawing a __polygon__. Country shapefiles can be downloaded at http://www.gadm.org/country.  
**Note:** The boundaries and names of the shapefiles do not imply official endorsement or acceptance by the United Nations.
3. Define the time period of your data: January __2000__ to __today__
4. Select the product: __MOD13Q1__ and the layers of interest: __EVI__ and __pixel_reliability__
5. Define the output as __Geotiff__ with __geographic Pojection__
6. Click on __Submit__

![](http://www.un-spider.org/sites/default/files/snap2_adj.PNG)

Click on __Explore__ to check the status of the request. When the status says "Done" you can click on the __Download__ symbol. __Select all__ ordered datasets and start downloading the data.

![](http://www.un-spider.org/sites/default/files/snap3_adj_new.PNG)

After the download store the files (.tif) in two folders: one folder for the __evi_data__ and one for the __pixel_reliability__.
It is reccomended to use an external hard drive to store your input and output data if possible. 
![](http://www.un-spider.org/sites/default/files/folders.PNG)

### Install packages

The modul __gdal__ needs to be installed separately and Bigtiffs must be supported. For that reason it is recommended to install gdal from __conda-forge__ (https://conda-forge.org/) as follows:
1. Open the __Anaconda Promt__ by searching for it in the windows start menu.
2. Type: `conda install -c conda-forge gdal` and hit `Enter`
![](http://www.un-spider.org/sites/default/files/gdal_inst.PNG)
3. Type `y` if you are asked if you want to proceed.

__Optional__: Create a virtual environment and install the package there if you plan to use python for different projects as some packages might interfere with each other. More information can be found __[here](https://github.com/yy/dviz-course/wiki/Create-and-manage-virtual-environments-with-Anaconda)__.

### Download this notebook and run it using Jupyter Notebook

Download the script (.ipynb) __[here](https://drive.google.com/drive/folders/1a3m5EbL2HGkkTLRA5U5V0oUH_J_V1DTT?usp=sharing)__ and save it on your computer. Start Jupyter Notebook by searching for it in the Windows start menu or by typing `jupyter notebook` in the Anaconda Prompt and hitting `Enter`. After Jupyter Notebook opened in your web browser, search for the script on your computer and open it.

Every cell has to be executed individually. A cell, which is still processing is marked by a `*` and a finished cell with a number. Cells are executed by hitting `Shift + Enter` or by clicking on the run button.

### Load packages

In [1]:
from osgeo import gdal
import os, osr
import numpy as np
gdal.UseExceptions()

In [2]:
import matplotlib.pyplot as plt
from matplotlib import colors
%matplotlib inline

In [3]:
# The script creates some warnings due to the NA in the data. They are ignored by executing this cell.
import warnings
warnings.filterwarnings('ignore')

### Set data paths 

Input data paths: ------------- ### TODO ### -------------

In [4]:
# Specify the name of your study area
studyarea = "Ghana"

In [5]:
# Specify the folder path where the EVI geotiffs are stored on your computer or external hard drive.
path_d = "D:/Ghana/evi_data/"

# Specify the folder path where the pixel reliability geotiffs are stored on your computer or external hard drive.
path_c = "D:/Ghana/pixel_reliability"

Output data paths: ------------- ### TODO ### -------------

In [6]:
# Create and specify the folder where the output png-files should be stored
path_png = "D:/Ghana/SVI_maps_Ghana_png"

# Create and specify the folder where the output tif-files should be stored
path_tif = "D:/Ghana/SVI_maps_Ghana_tif"

### Start processing

In the following cell, a function to write the data as geotiffs is defined. The geo information is extracted from the input data.

In [7]:
def array2raster(newRasterfn,array): 

    cols = array.shape[1]                                                         # Raeds columns from data array
    rows = array.shape[0]                                                         # Reads rows from data array

    driver = gdal.GetDriverByName('GTiff')                                        # Sets the driver to Geotiff 
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Float64)       # Creates raster with shape of array as float  
    outRaster.SetGeoTransform(expfile.GetGeoTransform())                          # Reads geo information from input file
    outband = outRaster.GetRasterBand(1)                                          # Reads band
    outband.SetNoDataValue(np.nan)                                                # Sets no data value to NA 
    outband.WriteArray(array) 
    outRaster.SetProjection(expfile.GetProjection())                           # Sets the projection according to the  
    outband.FlushCache()                                                       # input file projection

Within the following cell the Modis files are loaded per DOY (Day of the year), the cloudmask from the pixel reliability dataset  is applied, the EVI data is rescaled, the SVI is calculated and the SVI is written as smaller resolution PNG image and as Geotiff at the according output folders for every file.

__Note__: The script only works for files with file names according to the AppEEARS products, e.g. `MOD13Q1.006__250m_16_days_EVI_doy2001001_aid0001.tif` as the DOY information is extracted from the file name.

__Option__: If you want to use a different product (also geotiffs), you need to adapt the script where `file.split()` is used (three times). There, the files are split at the underscores and the 6th variable is used in this case to extract the DOY from the file name. If the DOY appears at a different position in your file name you need to adapt the number in the first squared brackets after the `file.split()`. Be aware that python uses zero-based indexing, so you need to start counting with zero.

In [9]:
for day in range(1, 365, 16):  # Creates a list of numbers between 0 and 365
    day = str('%0.3d' % day)   # Defines the structure of the list as 001, 002, ... equal to the DOY of the filename
    
    
    # Reading an example file and define the shape of the data arrays
    
    filelist =[]
    
    for root, dirs, files in os.walk(path_d):  # Creates a list of files, which include the defined day in their filename
         for file in files:                    # for the EVI data  
            if file.split("_")[6][7:] == day:
                filelist.append(file)
                
    expfile = gdal.Open(os.path.join(path_d, filelist[0]))  # Gets the first file of the file list as example file
    
    evi = np.zeros((expfile.RasterYSize, expfile.RasterXSize, len(filelist)))        # Creates 3D numpy arrays according to the 
    cloudmask = np.zeros((expfile.RasterYSize, expfile.RasterXSize, len(filelist)))  # example file dimensions 
    
    
    # Reading the cloudmask for all years 
    
    rellist =[]
    for root, dirs, files in os.walk(path_c):  # Creates a list of files, which include the defined day in their filename
        for file in files:                     # for the pixel reliability data  
            if file.split("_")[7][7:] == day:
                rellist.append(file)
    
    j = 0
    for file in rellist:
        dataset_c = gdal.Open(os.path.join(path_c, file))    # Opens the file from the pixel reliability file list
        band_c = dataset_c.GetRasterBand(1)
        data_c = band_c.ReadAsArray()
        dataset_c = None
        
        cloudmask[:,:,j] = data_c                         # Writes the data from each file to the array 
                                                          # dataset into the pre-defined 3D numpy array  
        j += 1
        del(data_c, band_c)                               # Deletes intermdiate products
    
        cloudmask[cloudmask == 0] = 1                     # Exchanges value 0 with value 1 (Pixel with value 0 and 1 are used)
        cloudmask[cloudmask == 2] = np.nan                # Sets value 2 to NA (Snow and Ice)
        cloudmask[cloudmask == 3] = np.nan                # Sets value 3 to NA (Cloud) 
        cloudmask[cloudmask == -1] = np.nan               # Sets no data value (= -1) to NA
        cloudmask[cloudmask > 3] = np.nan                 # Sets all values above 3 to NA
    
    
    # Reading the EVI  data
    
    i = 0
    for file in filelist:
        dataset = gdal.Open(os.path.join(path_d, file))      # Opens the file from the evi file list
        band = dataset.GetRasterBand(1)
        data = band.ReadAsArray()
        dataset = None
        
        data = data * cloudmask[:,:,i]                    # Applies the cloud mask
    
        data = data * 0.0001                              # Rescale the data (to float)
        data[data == -0.3] = np.nan                       # Sets no data value to NA
        data[data < 0] = np.nan                           # Sets negative values to NA    
        evi[:,:,i] = data                                 # Writes the data from each file to the array
        
        i += 1
        del(data, band)                                   # Deletes intermediate products

    del(cloudmask)
    
    
    # Calculating the SVI
    
    evimean = np.nanmean(evi, axis=2)                     # Calculates the mean of the array (NA are ignored)   
    evisd = np.nanstd(evi, axis=2)                        # Calculates the standard deviation of the array (NA are ignored) 
    
    svi = np.zeros((expfile.RasterYSize, expfile.RasterXSize, len(filelist)))  # Calculate the SVI 
    for k in range(len(filelist)):
        svi[:,:,k] = (evi[:,:,k]-evimean)/evisd
    
    del(evimean, evisd, evi)                             # Deletes intermediate products 
    
    
    
    # Creating png images and geotiffs as outputs
    
    for l in range(len(filelist)):                       # Generates png images for every time step 
        stdev = np.nanstd(svi[:,:,l])            # Calculates the standard deviation to define the color scheme 
        stdev2 = 2*stdev
        stdev15 = 1.5*stdev
        breaks = [-4, -stdev2, -stdev15, -stdev, stdev, stdev15, stdev2, 4]
        
        dayyear = day+filelist[l].split("_")[6][3:7]     # Defines the day and year of file for the output name of the image
        year = filelist[l].split("_")[6][3:7]
        
        fig, ax = plt.subplots(figsize=(5,5)) # Defines the size of the figure (in inches)
        plt.title("SVI (EVI) %s" %dayyear)     
        data = svi[:,:,l]
        cmap = colors.ListedColormap(['#4C0E0D', '#E72223', '#F19069', '#F9F6C6', '#64B14B', '#04984A', '#00320E'])
        bounds = breaks
        norm = colors.BoundaryNorm(bounds, cmap.N)
        cax = ax.imshow(data, cmap=cmap, norm=norm)
        fig.colorbar(cax, cmap=cmap, norm=norm, boundaries=bounds, ticks=breaks)
        plt.savefig(os.path.join(path_png, "SVI_"+studyarea+"_"+str(day)+"_"+str(year)+".png"), dpi=100)  # Saves png images in 
        plt.close()                                                                                       # according folder 
        
        
        # Generating geotiffs as defined above for every time step
    
        newRasterfn = os.path.join(path_tif, "SVI_"+studyarea+"_"+str(day)+"_"+str(year)+".tif")  # Saves geotiffs in according
        array = svi[:,:,l]                                                                        # folder
        
        array2raster(newRasterfn,array)    # Uses pre-defined function to write geotiffs
            
    del(svi)    # Deletes SVI

***

__Note__: Using a large area of interest can lead to troubles with the memory (RAM) of your computer. You can use the Task Manager to keep an eye on the memory usage while running the script. If you encounter problems, consider using a smaller area of interest (shapefile or polygon in AppEEARS) or a satellite product with a smaller resolution (500m (MOD13A1), 1km (MOD13A2)).

This is an example of the output PNG images. The geotiffs can be read by QGIS, for example, and used to produce maps.
![](http://www.un-spider.org/sites/default/files/SVI_Ghana_001_2007.png)